## Evaluate SV callsets of Manta, DELLY, LUMPY and GRIDSS based on two truth sets:

- Personalis/1000 Genomes Project data [(Parikh et al., 2016)](https://doi.org/10.1186%2Fs12864-016-2366-2),
- PacBio/Moleculo data [(Layer et al., 2014)](https://doi.org/10.1186/gb-2014-15-6-r84).


In [1]:
library(tools)
suppressPackageStartupMessages(require(StructuralVariantAnnotation))

### user input

In [2]:
sel.idx <- 1  # select one of the truth sets: 1 = Personalis1kGP or 2 = PacbioMoleculo

In [3]:
min.supp <- 3  # min. number of supporting callers

In [4]:
data.dir <- file.path('benchmark', 'out', '3', 'S3')  # with SV callsets

### store file names/paths

In [5]:
truth.sets <- list()
truth.sets$Personalis1kGP <- list(
    sv.file='Personalis_1000_Genomes_deduplicated_deletions.bed',
    excl.file='ENCFF001TDO.bed')
truth.sets$PacbioMoleculo <- list(
    sv.file='3717462611446476_add4.bedpe',
    excl.file='ceph18.b37.lumpy.exclude.2014-01-15.bed')
truth.sets <- lapply(truth.sets, lapply,
                     function(fn){file.path('benchmark', 'in', fn)})

### define helper functions

In [6]:
# get VCF file path given a caller
getVcf <- function(caller) {
    return(file.path(data.dir, paste0(caller, '_out'), paste0(caller, '.vcf')))
}

In [7]:
# assign SV types
# https://github.com/PapenfussLab/gridss/blob/7b1fedfed32af9e03ed5c6863d368a821a4c699f/example/simple-event-annotation.R#L9
getSvType <- function(gr) {
    return(ifelse(seqnames(gr) != seqnames(partner(gr)), 'CTX',
        ifelse(gr$insLen >= abs(gr$svLen) * 0.7, 'INS',
            ifelse(strand(gr) == strand(partner(gr)), 'INV',
                ifelse(xor(start(gr) < start(partner(gr)), strand(gr) == '-'),
                    'DEL', 'DUP')))))
}

In [8]:
# compute performance metrics for a callset
getPerfMetrics <- function(callset, hits, n.true) {
    n <- length(hits)
    tp <- sum(hits)
    fp <- n - tp
    fn <- n.true - tp
    prec <- round(tp * 100 / n, digits=1)
    rec <- round(tp * 100 / n.true, digits=1)
    return(list(callset=callset, n=n, tp=tp, fp=fp, fn=fn, precision=prec,
                recall=rec))
}

In [9]:
# exclude genomic regions
excludeRegions <- function(query.gr, subject.gr) {
    return(query.gr[!(overlapsAny(query.gr, subject.gr) |
                      overlapsAny(partner(query.gr), subject.gr)), ])
}

In [10]:
# count breakpoint overlaps (hits)
getHits <- function(query.gr, subject.gr) {
    return(countBreakpointOverlaps(query.gr, subject.gr, maxgap=100,
                                   sizemargin=0.25, ignore.strand=TRUE,
                                   restrictMarginToSizeMultiple=0.5,
                                   countOnlyBest=TRUE))
}

### convert BED to BEDPE file if applicable


In [11]:
bed.file <- truth.sets$Personalis1kGP$sv.file
bedpe.file <- paste0(file_path_sans_ext(bed.file), '.bedpe')
if(!file.exists(bedpe.file)) {
    cmd <- paste("awk 'BEGIN {a=0; OFS=\"\t\"} NR>1 {print $1,$2,$2+1,$1,$3,\
                 $3+1,\"DEL_\" a,-1,\"+\",\"+\",\"DEL\"; a+=1}'", bed.file, '>',
                 bedpe.file)
    system(cmd)
}

if(file.exists(bedpe.file)) {
    truth.sets$Personalis1kGP$sv.file <- bedpe.file
}

### import "true" deletions from BEDPE file

In [12]:
bedpe.file <- truth.sets[[sel.idx]]$sv.file
true.gr <- pairs2breakpointgr(rtracklayer::import(bedpe.file))
seqlevelsStyle(true.gr) <- 'NCBI'  # chr[X] -> [X]
min.svLen <- min(abs(end(partner(true.gr)) - start(true.gr)) + 1)
message('### Truth set ###')
message('input = ', bedpe.file)
message('n = ', length(true.gr))
message('min.svLen = ', min.svLen)

### Truth set ###
input = benchmark/in/Personalis_1000_Genomes_deduplicated_deletions.bedpe
n = 5352
min.svLen = 50


### filter the deletions by an exclusion list

In [13]:
bed.excl.file <- truth.sets[[sel.idx]]$excl.file
excl.gr <- rtracklayer::import(bed.excl.file)
seqlevelsStyle(excl.gr) <- 'NCBI'  # chr[X] -> X
message('\n### Exclusion list ###')
message('input = ', bed.excl.file)
print(seqnames(excl.gr))

true.gr <- excludeRegions(true.gr, excl.gr)
min.svlen <- min(abs(end(partner(true.gr)) - start(true.gr)) + 1)
n.true <- length(true.gr)
message('\n### Truth set filtered by the exclusion list ###')
message('n = ', n.true)
message('min.svLen = ', min.svLen)


### Exclusion list ###
input = benchmark/in/ENCFF001TDO.bed


factor-Rle of length 411 with 25 runs
  Lengths:  27  12  13   8   2   4   1  10 ...  14  10  18  13  23   1  25 118
  Values :   1  10  11  12  13  14  15  16 ...   5   6   7   8   9   M   X   Y
Levels(25): 1 10 11 12 13 14 15 16 17 18 19 2 20 21 22 3 4 5 6 7 8 9 M X Y



### Truth set filtered by the exclusion list ###
n = 5290
min.svLen = 50


### import SV callsets from VCF files

In [14]:
callers <- c('manta', 'delly', 'lumpy', 'gridss')
hits.df <- data.frame(callset=character(), n=numeric(), tp=numeric(),
                      fp=numeric(), precision=numeric(), recall=numeric())
for (c in callers) {
  vcf.file <- getVcf(c)
  vcf <- VariantAnnotation::readVcf(vcf.file)
  # select only DELs
  gr <- breakpointRanges(vcf)
  gr$svtype <- getSvType(gr)
  gr <- gr[gr$svtype == 'DEL']
  message('\n### All DELs ###')
  message('# ', c)
  message('n = ', length(gr))
  print(summary(gr$svLen))

  message('\n### DELs svLen != NA ###')
  gr <- gr[!is.na(gr$svLen)]
  message('# ', c)
  message('n = ', length(gr))
  print(summary(gr$svLen))

  gr <- gr[abs(gr$svLen) >= min.svLen]
  message('\n### DELs svLen >= min.svLen ###')
  message('# ', c)
  message('n = ', length(gr))
  print(summary(gr$svLen))

  gr <- excludeRegions(gr, excl.gr)
  message('\n### DELs >= min.svlen AND filtered by the exclusion list ###')
  message('# ', c)
  message('n = ', length(gr))
  print(summary(gr$svLen))

  hits <- getHits(gr, true.gr)
  pm <- getPerfMetrics(c, hits, n.true)
  hits.df <- rbind(hits.df, data.frame(pm))
}


### All DELs ###
# manta
n = 9214


     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-58995760      -339      -190    -41791       -75       -35 



### DELs svLen != NA ###
# manta
n = 9214


     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-58995760      -339      -190    -41791       -75       -35 



### DELs svLen >= min.svLen ###
# manta
n = 9150


...

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-122193018       -416       -274     -89061        -46        -16 



### DELs svLen >= min.svLen ###
# gridss
n = 5386


      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-122193018      -1069       -324    -123309       -172        -50 


Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': MT, GL000207.1, GL000226.1, GL000229.1, GL000231.1, GL000210.1, GL000239.1, GL000235.1, GL000201.1, GL000247.1, GL000245.1, GL000197.1, GL000203.1, GL000246.1, GL000249.1, GL000196.1, GL000248.1, GL000244.1, GL000238.1, GL000202.1, GL000234.1, GL000232.1, GL000206.1, GL000240.1, GL000236.1, GL000241.1, GL000243.1, GL000242.1, GL000230.1, GL000237.1, GL000233.1, GL000204.1, GL000198.1, GL000208.1, GL000191.1, GL000227.1, GL000228.1, GL000214.1, GL000221.1, GL000209.1, GL000218.1, GL000220.1, GL000213.1, GL000211.1, GL000199.1, GL000217.1, GL000216.1, GL000215.1, GL000205.1, GL000219.1, GL000224.1, GL000223.1, GL000195.1, GL000212.1, GL000222.1, GL000200.1, GL000193.1, GL000194.1, GL000225.1, GL000192.1, ERCC-00002, ERCC-00003, ERCC-00004, ERCC-00007, ERCC-00009, ERCC-00012, ERCC-00013, ERCC-00014, ERCC-00016, ERCC-00017, ERCC-00018, ERCC-00019, ERCC-00022, ERCC-00

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-122193018      -1069       -324    -123309       -172        -50 


### import merged callset from VCF file

In [15]:
# fix: replace ':' by '_' in ID & SAMPLE fields
vcf.infile <- file.path(data.dir, 'all.vcf')
vcf.outfile <- 'merge.vcf'
cmd <- paste("awk '{if ($1 ~ /^#/){print} else {id=$3; gsub(\":\",\"_\",$3);\
             gsub(id,$3,$10); print}}'", vcf.infile, '>', vcf.outfile)
system(cmd)

In [16]:
vcf <- VariantAnnotation::readVcf(vcf.outfile)
# fix: INFO/CI{END,POS} types: String->Integer
info(header(vcf))$Type[1:2] <- c("Integer", "Integer")
vcf <- vcf[which(info(vcf)$SVTYPE == 'DEL')]  # keep only deletions
vcf <- vcf[which(as.integer(info(vcf)$SUPP) >= min.supp, TRUE)]  # filter calls by support or
#vcf <- vcf[which(info(vcf)$SUPP_VEC == '1101', TRUE)]           # binary vector (MDLG)
# fix: CI{POS,END}} type: CharacterList->IntegerList
info(vcf)$CIPOS <- IntegerList(info(vcf)$CIPOS)
info(vcf)$CIEND <- IntegerList(info(vcf)$CIEND)

Warning message in doTryCatch(return(expr), name, parentenv, handler):
“record 5 sample S3.bam: fewer FORMAT fields than GENO fields”Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': 080418_Consensus_Vector_Sequence_NIST_SEQUENCING_ASSEMBLY_noRestrict_rev, ERCC-00002, ERCC-00003, ERCC-00004, ERCC-00007, ERCC-00009, ERCC-00012, ERCC-00013, ERCC-00014, ERCC-00016, ERCC-00017, ERCC-00018, ERCC-00019, ERCC-00022, ERCC-00023, ERCC-00024, ERCC-00025, ERCC-00028, ERCC-00031, ERCC-00033, ERCC-00034, ERCC-00035, ERCC-00039, ERCC-00040, ERCC-00041, ERCC-00042, ERCC-00043, ERCC-00044, ERCC-00046, ERCC-00048, ERCC-00051, ERCC-00053, ERCC-00054, ERCC-00057, ERCC-00058, ERCC-00059, ERCC-00060, ERCC-00061, ERCC-00062, ERCC-00067, ERCC-00069, ERCC-00071, ERCC-00073, ERCC-00074, ERCC-00075, ERCC-00076, ERCC-00077, ERCC-00078, ERCC-00079, ERCC-00081, ERCC-00083, ERCC-00084, ERCC-00085, ERCC-00086, ERCC-00092, ERCC-00095, ERCC-000

### evaluate SV break-points based on the truth set

In [17]:
gr <- breakpointRanges(vcf)
gr <- excludeRegions(gr, excl.gr)
hits <- getHits(gr, true.gr)
callset <- file_path_sans_ext(vcf.outfile)
pm <- getPerfMetrics(callset, hits, n.true)
hits.df <- rbind(hits.df, data.frame(pm))
message('\n### Performance metrics ###')
message('min.supp = ', min.supp, '\n')

Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:


  - in 'x': 080418_Consensus_Vector_Sequence_NIST_SEQUENCING_ASSEMBLY_noRestrict_rev, ERCC-00002, ERCC-00003, ERCC-00004, ERCC-00007, ERCC-00009, ERCC-00012, ERCC-00013, ERCC-00014, ERCC-00016, ERCC-00017, ERCC-00018, ERCC-00019, ERCC-00022, ERCC-00023, ERCC-00024, ERCC-00025, ERCC-00028, ERCC-00031, ERCC-00033, ERCC-00034, ERCC-00035, ERCC-00039, ERCC-00040, ERCC-00041, ERCC-00042, ERCC-00043, ERCC-00044, ERCC-00046, ERCC-00048, ERCC-00051, ERCC-00053, ERCC-00054, ERCC-00057, ERCC-00058, ERCC-00059, ERCC-00060, ERCC-00061, ERCC-00062, ERCC-00067, ERCC-00069, ERCC-00071, ERCC-00073, ERCC-00074, ERCC-00075, ERCC-00076, ERCC-00077, ERCC-00078, ERCC-00079, ERCC-00081, ERCC-00083, ERCC-00084, ERCC-00085, ERCC-00086, ERCC-00092, ERCC-00095, ERCC-00096, ERCC-00097, ERCC-00098, ERCC-00099, ERCC-00104, ERCC-00108, ERCC-00109, ERCC-00111, ERCC-00112, ERCC-00113, ERCC-00116, ERCC-00117, ERCC-00120, ERCC-00123, ERCC-00126, ERCC-00128, ERCC-00130, ERCC-00131, ERCC-00134, ERCC-00136, ERCC-00137, ER

  - in 'x': 080418_Consensus_Vector_Sequence_NIST_SEQUENCING_ASSEMBLY_noRestrict_rev, ERCC-00002, ERCC-00003, ERCC-00004, ERCC-00007, ERCC-00009, ERCC-00012, ERCC-00013, ERCC-00014, ERCC-00016, ERCC-00017, ERCC-00018, ERCC-00019, ERCC-00022, ERCC-00023, ERCC-00024, ERCC-00025, ERCC-00028, ERCC-00031, ERCC-00033, ERCC-00034, ERCC-00035, ERCC-00039, ERCC-00040, ERCC-00041, ERCC-00042, ERCC-00043, ERCC-00044, ERCC-00046, ERCC-00048, ERCC-00051, ERCC-00053, ERCC-00054, ERCC-00057, ERCC-00058, ERCC-00059, ERCC-00060, ERCC-00061, ERCC-00062, ERCC-00067, ERCC-00069, ERCC-00071, ERCC-00073, ERCC-00074, ERCC-00075, ERCC-00076, ERCC-00077, ERCC-00078, ERCC-00079, ERCC-00081, ERCC-00083, ERCC-00084, ERCC-00085, ERCC-00086, ERCC-00092, ERCC-00095, ERCC-00096, ERCC-00097, ERCC-00098, ERCC-00099, ERCC-00104, ERCC-00108, ERCC-00109, ERCC-00111, ERCC-00112, ERCC-00113, ERCC-00116, ERCC-00117, ERCC-00120, ERCC-00123, ERCC-00126, ERCC-00128, ERCC-00130, ERCC-00131, ERCC-00134, ERCC-00136, ERCC-00137, ER

### write performance metrics into CSV file

In [18]:
hits.df

callset,n,tp,fp,fn,precision,recall
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
manta,9084,4444,4640,846,48.9,84.0
delly,12042,4514,7528,776,37.5,85.3
lumpy,382,262,120,5028,68.6,5.0
gridss,5386,4134,1252,1156,76.8,78.1
merge,3660,3224,436,2066,88.1,60.9


In [19]:
csv.file <- paste0(names(truth.sets)[sel.idx], '_', 'metrics_supp-', min.supp, '.csv')
write.table(hits.df, file=csv.file, row.names=FALSE, col.names=TRUE,
            quote=FALSE, sep=',')